In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import scipy.stats

from pandas_datareader import data
from scipy.optimize import minimize

In [2]:
data_collection = pd.read_csv('Final_stock_data_V2.csv')
adjClose_5y = pd.read_csv('Cleaned_AdjCloseP_5y.csv', index_col='Date')
close_5y = pd.read_csv('Cleaned_CloseP_5y.csv', index_col='Date')


In [3]:
adjClose_5y = adjClose_5y[['AAP', 'ABT']]

In [4]:
tickers = adjClose_5y.columns

In [5]:
adjClose_5y.head()

,AAP,ABT
Date,,
2017-02-03,157.890411,39.270012
2017-02-06,155.484421,38.948734
2017-02-07,154.426956,39.370998
2017-02-08,157.870956,38.921196
2017-02-09,159.908234,39.049706


In [6]:
adjClose_5y.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1274 entries, 2017-02-03 to 2022-02-24
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype  
---  ------  --------------  -----  
 0   AAP     1274 non-null   float64
 1   ABT     1274 non-null   float64
dtypes: float64(2)
memory usage: 29.9+ KB


# ____________________________________________

In [7]:
risk_free_ann_ret_rate = 0.0176

In [8]:
returns_ts = adjClose_5y.pct_change().dropna()
returns_ts

,AAP,ABT
Date,,
2017-02-06,-0.015238,-0.008181
2017-02-07,-0.006801,0.010842
2017-02-08,0.022302,-0.011425
2017-02-09,0.012905,0.003302
2017-02-10,-0.004732,0.004702
...,...,...
2022-02-17,-0.008903,-0.025065
2022-02-18,-0.007305,-0.031431
2022-02-22,-0.055027,0.006165


In [9]:
avg_daily_ret = returns_ts.mean()
avg_daily_ret

AAP    0.000437
ABT    0.000993
dtype: float64

In [10]:
excess_ret = pd.DataFrame()
excess_ret

""


In [11]:

returns_ts['RiskFree_Rate'] = risk_free_ann_ret_rate/252
avg_rf_ret = returns_ts['RiskFree_Rate'].mean()
returns_ts

,AAP,ABT,RiskFree_Rate
Date,,,
2017-02-06,-0.015238,-0.008181,0.00007
2017-02-07,-0.006801,0.010842,0.00007
2017-02-08,0.022302,-0.011425,0.00007
2017-02-09,0.012905,0.003302,0.00007
2017-02-10,-0.004732,0.004702,0.00007
...,...,...,...
2022-02-17,-0.008903,-0.025065,0.00007
2022-02-18,-0.007305,-0.031431,0.00007
2022-02-22,-0.055027,0.006165,0.00007


In [12]:
for n in adjClose_5y.columns:
    excess_ret['Excess_ret_' + n] = returns_ts[n] - returns_ts['RiskFree_Rate']

In [13]:

def sharpe_ratio(excess_ret, ticker):

    sharpe_ratio = ((avg_daily_ret[ticker] - avg_rf_ret) / (excess_ret['Excess_ret_' + ticker].std())*np.sqrt(252))

    return sharpe_ratio.round(3)


In [14]:
sharpe_ratio_dic = {}

for n in tickers:

    sharpe_ratio_dic[n] = sharpe_ratio(excess_ret, n)

sharpe_ratio_dic

{'AAP': 0.263, 'ABT': 0.932}

In [16]:
n = len(adjClose_5y.columns)
log_ret = np.log(adjClose_5y / adjClose_5y.shift(1)).dropna()


def calculate_vol(weights):
    annualized_cov = np.dot(log_ret.cov()*252,weights)
    vol = np.dot(weights.transpose(),annualized_cov)
    return np.sqrt(vol)

def calculate_returns(weights, log_returns):
    # Annual log Returns

    return np.sum(log_returns.mean()*weights) * 252

def function_to_minimize(weights):

    # minimize a -1 * SR (that's how the math works here, but it is the same as maximizing SR)
    return -1 * ((calculate_returns(weights, log_ret) - risk_free_ann_ret_rate )/calculate_vol(weights)) 

# ---------------------

# Filter by Sector

In [ ]:
data_collection

In [ ]:
data_collection['sector'].hist(xrot=90)

# Test Profile => Growth
### Need ?
#### Sectors: Technology, Energy, real estate, consumer cyclical, communication services
#### High return
#### low correlation
#### max 40 stocks


In [ ]:
sectors = ['Technology', 'Energy', 'Real Estate', 'Consumer Cyclical', 'Communication Services']

In [ ]:
ind_data = data_collection[data_collection['sector'].isin(sectors)]
ind_data

In [ ]:
top_40_ret = ind_data['annu_return_5y'].sort_values(ascending=False)[:40]

In [ ]:
top_40_ret_data = ind_data.loc[top_40_ret.index]
top_40_ret_data

In [ ]:
top_40_ret_data['sector'].hist()

# TEST mini

In [ ]:
def calculate_returns(weights, log_returns):
    # Annual log Returns

    return np.sum(log_returns.mean()*weights) * 252

In [ ]:
n = len(adjClose_5y[top_40_ret_data['ticker']].columns)
log_returns_test = np.log(adjClose_5y[top_40_ret_data['ticker']] / adjClose_5y[top_40_ret_data['ticker']].shift(1)).dropna() # Same as 1 + stocks_data.pct_change()
log_returns_test

In [ ]:
def function_to_minimize(weights):

    # minimize a -1 * SR (that's how the math works here, but it is the same as maximizing SR)
    return -1 * ((calculate_returns(weights, log_returns_test) - 0 )/calculate_volatility(weights, log_returns_test)) # assuming rf =0


In [ ]:
required_return = 0.30
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
               {'type':'eq','fun': lambda weights: calculate_returns(weights,log_returns_test) - required_return})
bounds = tuple((0,1) for n in range(n))
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [ ]:
def calculate_vol(weights):
    annualized_cov = np.dot(log_returns_test.cov()*252,weights)
    vol = np.dot(weights.transpose(),annualized_cov)
    return np.sqrt(vol)

In [ ]:
result = minimize(fun=calculate_vol,x0=equal_weights,bounds=bounds,constraints=constraints)
result

In [ ]:
min_var_weights = result['x']
min_var_weights

In [ ]:
result['x']
portfolio = {}
p = 0
for i in result['x']:
    print('Put',(i*100).round(3), "% in", log_returns_test.columns[p])

    portfolio[log_returns_test.columns[p]] = (i*100).round(3)
    p += 1

In [ ]:
cleaned_portfolio = {}
for n in portfolio:
    if portfolio[n] != 0:
        cleaned_portfolio[n] = portfolio[n]
cleaned_portfolio

In [ ]:
list(cleaned_portfolio.keys())

In [ ]:
cleaned_pf_data = data_collection[data_collection['ticker'].isin(list(cleaned_portfolio.keys()))]
cleaned_pf_data

In [ ]:
cleaned_pf_data['sector'].hist()

# ___________________

In [ ]:
log_returns = np.log(adjClose_5y / adjClose_5y.shift(1)) # Same as 1 + stocks_data.pct_change()
log_returns

In [ ]:
n = len(adjClose_5y.columns)

In [ ]:
def gen_weights(n):
    weights = np.random.random(n)

    return weights / np.sum(weights)

In [ ]:
def calculate_returns(weights, log_returns):
    # Annual log Returns

    return np.sum(log_returns.mean()*weights) * 252

In [ ]:
def calculate_volatility(weights, log_returns):
    annualized_cov = np.dot(log_returns.cov()*252, weights)
    vol = np.dot(weights.transpose(), annualized_cov)

    return np.sqrt(vol)

In [ ]:
calculate_volatility(gen_weights(n), log_returns)

In [ ]:
mc_portfolio_returns = []
mc_portfolio_vol = []
mc_weights = []

for sim in range(20000):

    weights = gen_weights(n)
    mc_weights.append(weights)

    mc_portfolio_returns.append(calculate_returns(weights, log_returns))

    mc_portfolio_vol.append(calculate_volatility(weights, log_returns))



In [ ]:
mc_sharpe_ratio = (np.array(mc_portfolio_returns) - 0 )/np.array(mc_portfolio_vol) # assuming rf =0

In [ ]:
plt.figure(figsize = (10,5))
plt.scatter(mc_portfolio_vol, mc_portfolio_returns, c=mc_sharpe_ratio)
plt.colorbar(label='Sharpe Ratio')
plt.xlabel('Volatility')
plt.ylabel('Returns')

plt.show()


# ------------------------------

In [ ]:
mc_weights[np.argmax(mc_sharpe_ratio)]


['AAPL', 'ARKK', 'TSLA', 'AMZN', 'MSFT', 'GOOGL', 'JPM', 'NFLX']
array([0.02920397, 0.02217007, 0.07986235, 0.40333344, 0.281858  ,
       0.05538882, 0.04158916, 0.08659419])

array([0.16056466, 0.0022723 , 0.11224832, 0.33140994, 0.16490515,
       0.02462714, 0.02794456, 0.17602793])

# Max SH

In [ ]:
def function_to_minimize(weights):

    # minimize a -1 * SR (that's how the math works here, but it is the same as maximizing SR)
    return -1 * ((calculate_returns(weights, log_returns) - 0 )/calculate_volatility(weights, log_returns)) # assuming rf =0


In [ ]:
bounds = tuple((0,1) for n in range(n))
sum_constraint = ({'type': 'eq', 'fun': lambda weights: np.sum(weights)-1}) # Read the documentation for minimize for more details
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [ ]:
results = minimize(fun=function_to_minimize,
        x0=equal_weights,
        bounds=bounds,
        constraints=sum_constraint)

In [ ]:
results

In [ ]:
# weights for optimal portfolio using historical data SR
results['x']
p = 0
for i in results['x']:
    print('Put',(i*100).round(3), "% in", stocks_data.columns[p])
    p += 1

# Min Var for given return

In [ ]:
required_return = 0.4

In [ ]:
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
               {'type':'eq','fun': lambda weights: calculate_returns(weights,log_returns) - required_return})

In [ ]:
bounds = tuple((0,1) for n in range(n))
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [ ]:
def calculate_vol(weights):
    annualized_cov = np.dot(log_returns.cov()*252,weights)
    vol = np.dot(weights.transpose(),annualized_cov)
    return np.sqrt(vol)

In [ ]:
result = minimize(fun=calculate_vol,x0=equal_weights,bounds=bounds,constraints=constraints)

In [ ]:
result


In [ ]:
min_var_weights = result['x']

In [ ]:
min_var_weights

In [ ]:
calculate_vol(min_var_weights)

# Max return for given vol

In [ ]:
required_vol = 0.4

In [ ]:
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
               {'type':'eq','fun': lambda weights: calculate_vol(weights) - required_vol})

In [ ]:
bounds = tuple((0,1) for n in range(n))
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [ ]:
def calculate_ret(weights):
    # Annual log Returns

    return -1 * np.sum(log_returns.mean()*weights) * 252

In [ ]:
result = minimize(fun=calculate_ret,x0=equal_weights,bounds=bounds,constraints=constraints)

In [ ]:
result

In [ ]:
calculate_ret(result['x']) * -1

In [ ]:
given_ret = 0.3
given_vol = 0.4

In [ ]:
constraints = ({'type':'eq','fun': lambda weights: np.sum(weights)-1},
               {'type':'eq','fun': lambda weights: calculate_vol(weights) - given_vol},
               {'type':'eq','fun': lambda weights: calculate_returns(weights,log_returns) - given_ret})

In [ ]:
bounds = tuple((0,1) for n in range(n))
equal_weights = n * [1/n] #going to be our starting point then the function will look at either direction for minimum negative SG

In [ ]:
def function_to_minimize(weights):

    # minimize a -1 * SR (that's how the math works here, but it is the same as maximizing SR)
    return -1 * ((calculate_returns(weights, log_returns) - 0 )/calculate_volatility(weights, log_returns)) # assuming rf =0


In [ ]:
results = minimize(fun=function_to_minimize,
        x0=equal_weights,
        bounds=bounds,
        constraints=constraints)

In [ ]:
results